<a href="https://colab.research.google.com/github/DreRnc/ExplainingExplanations/blob/main/Explanations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset : **E-SNLI**. \
Model : **Small T5**.

In [1]:
colab = True

In [ ]:
if colab:
    !git clone https://github.com/DreRnc/ExplainingExplanations.git
    %cd ExplainingExplanations
    %pip install -r requirements.txt
    !git checkout seq2seq

# 1.0 Preparation


In [1]:
N_TRAIN = 1000
N_VAL = 1000
N_TEST = 1000

NUM_EPOCHS = 5

## 1.1 Loading Dataset

In [2]:
from datasets import load_dataset

dataset = load_dataset("esnli", download_mode="force_redownload")

/Users/biancaziliotto/Library/Caches/pypoetry/virtualenvs/explainingexplanations-VLxOqKFf-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 9824/9824 [00:00<00:00, 2303361.98 examples/s]


In [3]:
training_set = dataset["train"]
validation_set = dataset["validation"]
test_set = dataset["test"]

print("Shape of training_set: ", training_set.shape)
print("Shae of validation_set: ", validation_set.shape)
print("Shape of test_set: ", test_set.shape)

Shape of training_set:  (549367, 6)
Shae of validation_set:  (9842, 6)
Shape of test_set:  (9824, 6)


In [4]:
training_set[0]

{'premise': 'A person on a horse jumps over a broken down airplane.',
 'hypothesis': 'A person is training his horse for a competition.',
 'label': 1,
 'explanation_1': 'the person is not necessarily training his horse',
 'explanation_2': '',
 'explanation_3': ''}

In [5]:
train_small = training_set.select(range(N_TRAIN))
valid_small = validation_set.select(range(N_VAL))
test_small = test_set.select(range(N_TEST))

print("Shape of train_small: ", train_small.shape)
print("Shape of valid_small: ", valid_small.shape)
print("Shape of test_small: ", test_small.shape)

Shape of train_small:  (1000, 6)
Shape of valid_small:  (1000, 6)
Shape of test_small:  (1000, 6)


## 1.2 Loading T5 Model

In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small", truncation=True, padding=True)
model = T5ForConditionalGeneration.from_pretrained("t5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Test **zero-shot** on a random task.

In [7]:
input_ids = tokenizer(
    "translate English to French: Hello Dre, I think the English version is ok for us.",
    return_tensors="pt",
).input_ids
outputs = model.generate(input_ids, max_length=20)

print(tokenizer.decode(outputs[0], skip_special_tokens=True, max_length=20))

Bonjour Dre, je pense que la version anglaise est bonne pour nous.


## 1.3 Zero-shot example to Verify Everything is Working

In [8]:
from src.utils import generate_prompt_mnli

In [9]:
example = training_set[0]
example

{'premise': 'A person on a horse jumps over a broken down airplane.',
 'hypothesis': 'A person is training his horse for a competition.',
 'label': 1,
 'explanation_1': 'the person is not necessarily training his horse',
 'explanation_2': '',
 'explanation_3': ''}

Generating the prompt:

<b><u> mnli hypothesis: </b></u> The St. Louis Cardinals have always won. <b><u> premise: </b></u> yeah well losing is i mean i’m i’m originally from Saint Louis and Saint Louis Cardinals when they were there were uh a mostly a losing team but

Output:
* 0: Entailment
* 1: Neutral
* 2: Contradiction

In [10]:
prompt = generate_prompt_mnli(example)
prompt

'mnli hypothesis: A person is training his horse for a competition. premise: A person on a horse jumps over a broken down airplane.'

In [11]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(outputs)
print("Shape of outputs:", outputs.shape)
print("Shape of outputs[0]:", outputs[0].shape)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

tensor([[   0, 7163,    1]])
Shape of outputs: torch.Size([1, 3])
Shape of outputs[0]: torch.Size([3])
neutral


/Users/biancaziliotto/Library/Caches/pypoetry/virtualenvs/explainingexplanations-VLxOqKFf-py3.10/lib/python3.10/site-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


## 1.4  Tokenize the dataset

In [12]:
train_small.info

DatasetInfo(description='', citation='', homepage='', license='', features={'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None), 'explanation_1': Value(dtype='string', id=None), 'explanation_2': Value(dtype='string', id=None), 'explanation_3': Value(dtype='string', id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name='parquet', dataset_name='esnli', config_name='plain_text', version=0.0.2, splits={'train': SplitInfo(name='train', num_bytes=107611996, num_examples=549367, shard_lengths=None, dataset_name='esnli'), 'validation': SplitInfo(name='validation', num_bytes=3416319, num_examples=9842, shard_lengths=None, dataset_name='esnli'), 'test': SplitInfo(name='test', num_bytes=3379781, num_examples=9824, shard_lengths=None, dataset_name='esnli')}, download_checksums={'hf://datasets/esnli@f01d1430c0ad65a3785d06079a0e01e015e769b8/plain_text/

In [13]:
train_small.features

{'premise': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None),
 'explanation_1': Value(dtype='string', id=None),
 'explanation_2': Value(dtype='string', id=None),
 'explanation_3': Value(dtype='string', id=None)}

In [14]:
from functools import partial
from src.utils import tokenize_function

In [15]:
tokenize_mapping = partial(tokenize_function, tokenizer=tokenizer)

In [16]:
train_small_tokenized = train_small.map(tokenize_mapping, batched=True).with_format(
    "torch"
)
valid_small_tokenized = valid_small.map(tokenize_mapping, batched=True).with_format(
    "torch"
)
test_small_tokenized = test_small.map(tokenize_mapping, batched=True).with_format(
    "torch"
)

print("Shape of train_small_tokenized: ", train_small_tokenized.shape)
print("Shape of valid_small_tokenized: ", valid_small_tokenized.shape)
print("Shape of test_small_tokenized: ", test_small_tokenized.shape)

Map: 100%|██████████| 1000/1000 [00:00<00:00, 11517.15 examples/s]

Shape of train_small_tokenized:  (1000, 9)
Shape of valid_small_tokenized:  (1000, 9)
Shape of test_small_tokenized:  (1000, 9)


In [17]:
train_small_tokenized = train_small_tokenized.remove_columns(["label"])
valid_small_tokenized = valid_small_tokenized.remove_columns(["label"])
test_small_tokenized = test_small_tokenized.remove_columns(["label"])

In [18]:
train_small_tokenized.features

{'premise': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'explanation_1': Value(dtype='string', id=None),
 'explanation_2': Value(dtype='string', id=None),
 'explanation_3': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

# 2.0 Tasks

In [25]:
import torch
from functools import partial
import evaluate
from src.utils import compute_metrics, eval_pred_transform_accuracy
from transformers import (
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    T5ForConditionalGeneration,
    DataCollatorForSeq2Seq,
)

In [26]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
device

device(type='cpu')

In [27]:
transform_accuracy = partial(eval_pred_transform_accuracy, tokenizer=tokenizer)
compute_accuracy = partial(
    compute_metrics, pred_transform=transform_accuracy, metric=evaluate.load("accuracy")
)

## 2.1 Task 1: Zero-shot evaluation

In [28]:
model = T5ForConditionalGeneration.from_pretrained("t5-small")
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [29]:
training_args = Seq2SeqTrainingArguments(
    output_dir="task1",
    predict_with_generate=True,
    per_device_eval_batch_size=16,
    generation_max_length=32,
    metric_for_best_model="accuracy",
)

In [30]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_small_tokenized,
    eval_dataset=valid_small_tokenized,
    compute_metrics=compute_accuracy,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.evaluate(test_small_tokenized)

## 2.2 Task 2: Fine tuning without explanations

In [ ]:
model_ft = T5ForConditionalGeneration.from_pretrained("t5-small")
data_collator_ft = DataCollatorForSeq2Seq(tokenizer, model=model_ft)

In [ ]:
training_args_ft = Seq2SeqTrainingArguments(
    output_dir="task2",
    evaluation_strategy="epoch",
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    generation_max_length=32,
    metric_for_best_model="accuracy",
)

In [ ]:
trainer_ft = Seq2SeqTrainer(
    model=model_ft,
    args=training_args_ft,
    train_dataset=train_small_tokenized,
    eval_dataset=valid_small_tokenized,
    compute_metrics=compute_accuracy,
    data_collator=data_collator_ft,
    tokenizer=tokenizer,
)

In [ ]:
trainer_ft.train()

In [ ]:
trainer_ft.evaluate(test_small_tokenized)

## 2.3 Task 3: Fine Tuning with Explanations

We need to give as labels the label and the explanation tokenized.

### Preparing the dataset with labelled explanations

In [ ]:
dataset_explanations = load_dataset("esnli", download_mode="force_redownload")

In [ ]:
training_set_ex = dataset_explanations["train"]
validation_set_ex = dataset_explanations["validation"]
test_set_ex = dataset_explanations["test"]

print("Shape of training_set: ", training_set_ex.shape)
print("Shae of validation_set: ", validation_set_ex.shape)
print("Shape of test_set: ", test_set_ex.shape)

In [ ]:
train_small_ex = training_set_ex.select(range(N_TRAIN))
valid_small_ex = validation_set_ex.select(range(N_VAL))
test_small_ex = test_set_ex.select(range(N_TEST))

print("Shape of train_small: ", train_small_ex.shape)
print("Shape of valid_small: ", valid_small_ex.shape)
print("Shape of test_small: ", test_small_ex.shape)

#### Tokenizing the dataset

In [ ]:
from functools import partial
from src.utils import tokenize_function_ex

In [ ]:
tokenize_mapping_ex = partial(tokenize_function_ex, tokenizer=tokenizer)

In [ ]:
train_small_tokenized_ex = train_small_ex.map(
    tokenize_mapping_ex, batched=True
).with_format("torch")
valid_small_tokenized_ex = valid_small_ex.map(
    tokenize_mapping_ex, batched=True
).with_format("torch")
test_small_tokenized_ex = test_small_ex.map(
    tokenize_mapping_ex, batched=True
).with_format("torch")

print("Shape of train_small_tokenized: ", train_small_tokenized_ex.shape)
print("Shape of valid_small_tokenized: ", valid_small_tokenized_ex.shape)
print("Shape of test_small_tokenized: ", test_small_tokenized_ex.shape)

In [ ]:
train_small_tokenized_ex = train_small_tokenized_ex.remove_columns(["label"])
valid_small_tokenized_ex = valid_small_tokenized_ex.remove_columns(["label"])
test_small_tokenized_ex = test_small_tokenized_ex.remove_columns(["label"])

In [ ]:
train_small_tokenized_ex.features

### Fine Tuning

In [ ]:
transform_accuracy_ex = partial(
    eval_pred_transform_accuracy,
    tokenizer=tokenizer,
    remove_explanations_from_label=True,
)
compute_accuracy_removing_explanations = partial(
    compute_metrics,
    pred_transform=transform_accuracy_ex,
    metric=evaluate.load("accuracy"),
)

In [ ]:
model_ft_ex = T5ForConditionalGeneration.from_pretrained("t5-small")
data_collator_ft_ex = DataCollatorForSeq2Seq(tokenizer, model=model_ft_ex)

In [ ]:
training_args_ft_ex = Seq2SeqTrainingArguments(
    output_dir="task3",
    evaluation_strategy="epoch",
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    generation_max_length=128,
    metric_for_best_model="accuracy",
)

In [ ]:
trainer_ft_ex = Seq2SeqTrainer(
    model=model_ft_ex,
    args=training_args_ft_ex,
    train_dataset=train_small_tokenized_ex,
    eval_dataset=valid_small_tokenized_ex,
    compute_metrics=compute_accuracy_removing_explanations,
    data_collator=data_collator_ft_ex,
    tokenizer=tokenizer,
)

In [ ]:
trainer_ft_ex.train()

In [ ]:
trainer_ft_ex.evaluate(test_small_tokenized_ex)

## 2.4 Task 4: Fine Tuning with Shuffled Explanations

### Preparing the dataset with *wrong* labelled explanations

In [34]:
texts = []
for example in train_small:  # You may need to change 'train' to the appropriate split
    texts.append(example["explanation_1"])

# Save the texts to a text file
with open("explainations.txt", "w", encoding="utf-8") as f:
    for text in texts:
        f.write(text + "\n")

In [35]:
import random

# Define the input and output file paths
input_file = "explainations.txt"
output_file = "shuffled_explainations.txt"

# Read the lines from the input file
with open(input_file, "r") as f:
    lines = f.readlines()

# Shuffle the lines
random.shuffle(lines)

# Write the shuffled lines to the output file
with open(output_file, "w") as f:
    f.writelines(lines)

In [36]:
with open("shuffled_explainations.txt", "r") as f:
    explanations = f.readlines()

In [37]:
from src.utils import tokenize_function_ex

tokenize_mapping = partial(
    tokenize_function_ex, tokenizer=tokenizer, modified_explanations=explanations
)

In [44]:
train_small_tokenized_shex = train_small.map(
    tokenize_mapping, batched=True
).with_format("torch")
valid_small_tokenized_shex = valid_small.map(
    tokenize_mapping, batched=True
).with_format("torch")
test_small_tokenized_shex = test_small.map(tokenize_mapping, batched=True).with_format(
    "torch"
)

print("Shape of train_small_tokenized: ", train_small_tokenized.shape)
print("Shape of valid_small_tokenized: ", valid_small_tokenized.shape)
print("Shape of test_small_tokenized: ", test_small_tokenized.shape)

Shape of train_small_tokenized:  (1000, 8)
Shape of valid_small_tokenized:  (1000, 8)
Shape of test_small_tokenized:  (1000, 8)


In [46]:
train_small_tokenized_shex = train_small_tokenized_shex.remove_columns(["label"])
valid_small_tokenized_shex = valid_small_tokenized_shex.remove_columns(["label"])
test_small_tokenized_shex = test_small_tokenized_shex.remove_columns(["label"])

### Fine Tuning

In [47]:
transform_accuracy_ex = partial(
    eval_pred_transform_accuracy,
    tokenizer=tokenizer,
    remove_explanations_from_label=True,
)
compute_accuracy_removing_explanations = partial(
    compute_metrics,
    pred_transform=transform_accuracy_ex,
    metric=evaluate.load("accuracy"),
)

In [48]:
model_ft_ex = T5ForConditionalGeneration.from_pretrained("t5-small")
data_collator_ft_ex = DataCollatorForSeq2Seq(tokenizer, model=model_ft_ex)

In [49]:
training_args_ft_ex = Seq2SeqTrainingArguments(
    output_dir="task3",
    evaluation_strategy="epoch",
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    generation_max_length=128,
    metric_for_best_model="accuracy",
)

In [51]:
trainer_ft_ex = Seq2SeqTrainer(
    model=model_ft_ex,
    args=training_args_ft_ex,
    train_dataset=train_small_tokenized_shex,
    eval_dataset=valid_small_tokenized_shex,
    compute_metrics=compute_accuracy_removing_explanations,
    data_collator=data_collator_ft_ex,
    tokenizer=tokenizer,
)

In [ ]:
trainer_ft_ex.train()

In [ ]:
trainer_ft_ex.evaluate(test_small_tokenized_ex)

## 2.5 Task 5: Profiling-UD